In [1]:
# Imports
import csv
import os
import pandas as pd
from django.core.exceptions import ValidationError
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from datetime import date
from edc_appointment.models import Appointment

In [3]:
data_path = '/Users/imosweu/Downloads/FLOURSIH_VISIT_TRACKING_4.csv'
# data_path = '/home/flourish/FLOURSIH_VISIT_TRACKING_4.csv'
data = []
with open(data_path) as td_csv:
    csvreader = csv.reader(td_csv)
    for row in csvreader:
        data.append(row)

In [4]:
# data

In [5]:
data.pop(0)

['\ufeffPARTICIPANTS',
 'MATERNAL HIV STATUS',
 'HIV TEST DATE',
 'COHORT',
 'Date Consented',
 'IS THE PARTICIPANT PREGNANT ?',
 'DID THE PARTICIPANT CONSENT  WITH V2?',
 'LNMP',
 'GA AT CONSENTING',
 'GA AT 1000 VISIT',
 "TODAY'S DATE",
 'EDC  CONFIRMED GA',
 'EDC CONFIRMED EDD',
 'GA TODAY\n(for ANC call from 35 weeks)',
 'COMMENTS',
 'DATE OF ANC ENROLMENT 1000',
 'WAS 1000 ATTENDED',
 'ACTUAL DATE OF DELIVERY',
 'DATE OF 2000 visit',
 'WAS 2000  VISIT ATTENDED?',
 'EARLIEST DATE DUE FOR Q1 VISIT-2001',
 'IDEAL DATE DUE FOR Q1 VISIT-2001',
 'LATEST DATE DUE FOR Q1 VISIT-2001',
 'SCHEDULED DATE FOR Q1 VISIT-2001',
 'WAS Q1 ATTENDED?-2001',
 'EARLIEST DATE DUE FOR Q2 VISIT -2002',
 'IDEAL DATE DUE FOR Q2 VISIT-2002',
 'LATEST DATE DUE FOR Q2 VISIT-2002',
 'SCHEDULED DATE FOR Q2 VISIT-2002',
 'WAS Q2 ATTENDED?2002',
 'EARLIEST DATE DUE FOR Q3 VISIT-2003',
 'IDEAL DATE DUE FOR Q3 VISIT-2003',
 'LATEST DATE DUE FOR Q3 VISIT-2003',
 'SCHEDULED DATE FOR Q3 VISIT-2003',
 'WAS Q3 ATTENDED?-

In [6]:
from dateutil import parser
from edc_appointment.models import Appointment
mismatch = []
mismatch_birth = []
missing = []
multi = []

for a in data:
    subject_identifier = a[0]
    if a[18] and 'Off' not in a[18] and 'OFF' not in a[18]:
        visit_date = parser.parse(a[18]).date()
    
    if visit_date and subject_identifier:
        if subject_identifier[0].upper() in ['B', 'C']:
            subject_identifier = subject_identifier[1:]
        
        #Get Appointment to check if its not new
        try:
            Appointment.objects.get(Q(appt_status='new'),
                                    subject_identifier__endswith=subject_identifier,
                                    visit_code__startswith='2000', visit_code_sequence=0,)
        except Appointment.DoesNotExist:
            #Get 2000M/2000D visit
            try:
                v1 = MaternalVisit.objects.get(subject_identifier__endswith=subject_identifier,
                                               visit_code__startswith='2000', visit_code_sequence=0)
            except MaternalVisit.DoesNotExist:
                #if 2000 visit not found,check if its an antenatal enrollement
                try:
                    MaternalVisit.objects.get(subject_identifier__endswith=subject_identifier,
                                               visit_code__startswith='1000', visit_code_sequence=0)
                except MaternalVisit.DoesNotExist:
                    missing.append(subject_identifier)
            except MaternalVisit.MultipleObjectsReturned:
                multi.append(subject_identifier)
            else:
                if v1.report_datetime.date() != visit_date:
                    if 'D' in v1.visit_code:
                        mismatch_birth.append([subject_identifier, visit_date, 
                                               v1.report_datetime.date(), v1.visit_code])
                    else:
                        mismatch.append([subject_identifier, visit_date, 
                                         v1.report_datetime.date(), v1.visit_code])

In [9]:
import datetime
from django.utils.timezone import make_aware
from django.conf import settings
from edc_appointment.models import Appointment

shocked = []

cohort_dict = {'cohort_a': OnScheduleCohortAQuarterly,
               'cohort_a_sec': OnScheduleCohortASecQuart,
               'cohort_b': OnScheduleCohortBQuarterly,
               'cohort_b_sec': OnScheduleCohortBSecQuart,
               'cohort_c': OnScheduleCohortCQuarterly,
               'cohort_c_sec': OnScheduleCohortCSecQuart
    }

# a = ['142-040990121-1', datetime.date(2021, 8, 11), datetime.date(2021, 8, 12), '2000M']
for a in mismatch:

    try:
        mv = MaternalVisit.objects.get(subject_identifier__endswith=a[0], visit_code=a[3], report_datetime__date=a[2])
    except MaternalVisit.DoesNotExist:
        shocked.append(a[0])
    else:
        if 'sec' in mv.schedule_name:
            x_name = 'cohort_' + mv.schedule_name[:5]
            vs = mv.schedule_name.split('_')
            schedule_name = '_'.join([vs[0], 'sec',  vs[1].replace('sec', 'quart'), vs[2]])
        else:
            x_name = 'cohort_' + mv.schedule_name[:1]
            vs = mv.schedule_name.split('_')
            schedule_name = '_'.join([vs[0], vs[1].replace('enrol', 'quarterly'), vs[2]])

        schedule_cls = cohort_dict.get(x_name)

        print(mv.subject_identifier)
        print(schedule_name)
        print('\n\n')

    #     import pdb; pdb.set_trace()
        onschedule_obj = schedule_cls.objects.get(subject_identifier=mv.subject_identifier, 
                                                 schedule_name=schedule_name)

        app_not_new = Appointment.objects.filter(subject_identifier=mv.subject_identifier, 
                                                 schedule_name=schedule_name).exclude(appt_status='new')
        for y in app_not_new:
            y.schedule_name = 'x'
            y.save()

        onschedule_obj.schedule_name = 'x'
        onschedule_obj.save()
        
        settings.TIME_ZONE  # 'UTC'

        mv.report_datetime = make_aware(datetime.datetime.combine(a[1], 
                            mv.report_datetime.time()))
        mv.save()

        for r in app_not_new:
            g = Appointment.objects.get(subject_identifier=r.subject_identifier, visit_code=r.visit_code,
                                       schedule_name=schedule_name)
            g.appt_status = r.appt_status
            g.appt_datetime = r.appt_datetime
            g.appt_reason = r.appt_reason
            g.comment = r.comment
            g.save()

            try:
                mm = MaternalVisit.objects.get(appointment=r)
            except MaternalVisit.DoesNotExist:
                pass
            else:
                mm.appointment = g
                mm.save_base(raw=True)

            r.delete()

        onschedule_obj.delete()



B142-040990046-0
b_sec_quart1_schedule1



B142-040990048-6
b_sec_quart1_schedule1



B142-040990082-5
b_sec_quart1_schedule1



B142-040990084-1
c_sec_quart1_schedule1



B142-040990121-1
c_quarterly1_schedule1



B142-040990135-1
b_quarterly1_schedule1



B142-040990321-7
b_quarterly1_schedule1



B142-040990324-1
b_quarterly1_schedule1



B142-040990351-4
b_sec_quart1_schedule1



B142-040990581-6
a_quarterly1_schedule1



B142-040990595-6
b_sec_quart1_schedule1



B142-040990606-1
c_sec_quart1_schedule1



B142-040990612-9
c_sec_quart1_schedule1





In [7]:
shocked

[]

In [10]:
#Birth Quart Fix
import datetime
from django.utils.timezone import make_aware
from django.conf import settings
from edc_appointment.models import Appointment

shocked = []

# a = ['142-040990121-1', datetime.date(2021, 8, 11), datetime.date(2021, 8, 12), '2000M']
for a in mismatch_birth:
        
        try:
            mv = MaternalVisit.objects.get(subject_identifier__endswith=a[0],
                                           visit_code=a[3], visit_code_sequence=0, report_datetime__date=a[2])
        except MaternalVisit.DoesNotExist:
            shocked.append(a[0])
            
        if mv.subject_identifier != 'B142-040990611-1':
            print(mv.subject_identifier)
            print('\n\n')

            onschedule_obj = OnScheduleCohortAQuarterly.objects.get(subject_identifier=mv.subject_identifier)

            schedule_name = onschedule_obj.schedule_name

            appts = Appointment.objects.filter(subject_identifier=mv.subject_identifier, 
                                               schedule_name=schedule_name)
            for y in appts:
                y.schedule_name = 'x'
                y.save()

            onschedule_obj.schedule_name = 'x'
            onschedule_obj.save()

            settings.TIME_ZONE  # 'UTC'

            mv.report_datetime = make_aware(datetime.datetime.combine(a[1], 
                                mv.report_datetime.time()))
            mv.save()

            for r in appts:
                g = Appointment.objects.get(subject_identifier=r.subject_identifier, visit_code=r.visit_code,
                                           schedule_name=schedule_name)
                if r.appt_status != 'new':
                    g.appt_status = r.appt_status
                    g.appt_datetime = r.appt_datetime
                    g.appt_reason = r.appt_reason
                    g.comment = r.comment
                    g.save()

                try:
                    mm = MaternalVisit.objects.get(appointment=r)
                except MaternalVisit.DoesNotExist:
                    pass
                else:
                    mm.appointment = g
                    mm.save_base(raw=True)

                r.delete()

            onschedule_obj.delete()



B142-040990385-2



B142-040990391-0



B142-040990436-3



B142-040990508-9



B142-040990550-1



B142-040990553-5



B142-040990555-0



B142-040990572-5



B142-040990668-1





In [18]:
# #Child Enroll
# import datetime
# from django.utils.timezone import make_aware
# from django.conf import settings
# from flourish_child.models import Appointment

# shocked = []

# cohort_dict = {'cohort_a': OnScheduleCohortAQuarterly,
#                'cohort_a_sec': OnScheduleCohortASecQuart,
#                'cohort_b': OnScheduleCohortBQuarterly,
#                'cohort_b_sec': OnScheduleCohortBSecQuart,
#                'cohort_c': OnScheduleCohortCQuarterly,
#                'cohort_c_sec': OnScheduleCohortCSecQuart
#     }

# # a = ['142-040990121-1', datetime.date(2021, 8, 11), datetime.date(2021, 8, 12), '2000M']
# for a in mismatch:

#     try:
#         mv = MaternalVisit.objects.get(subject_identifier__endswith=a[0], visit_code=a[3], report_datetime__date=a[2])
#     except MaternalVisit.DoesNotExist:
#         shocked.append(a[0])
#     else:
#         if 'sec' in mv.schedule_name:
#             x_name = 'cohort_' + mv.schedule_name[:5]
#             vs = mv.schedule_name.split('_')
#             schedule_name = '_'.join([vs[0], 'sec',  vs[1].replace('sec', 'quart'), vs[2]])
#         else:
#             x_name = 'cohort_' + mv.schedule_name[:1]
#             vs = mv.schedule_name.split('_')
#             schedule_name = '_'.join([vs[0], vs[1].replace('enrol', 'quarterly'), vs[2]])

#         schedule_cls = cohort_dict.get(x_name)

#         print(mv.subject_identifier)
#         print(schedule_name)
#         print('\n\n')

#     #     import pdb; pdb.set_trace()
#         onschedule_obj = schedule_cls.objects.get(subject_identifier=mv.subject_identifier, 
#                                                  schedule_name=schedule_name)

#         app_not_new = Appointment.objects.filter(subject_identifier=mv.subject_identifier, 
#                                                  schedule_name=schedule_name).exclude(appt_status='new')
#         for y in app_not_new:
#             y.schedule_name = 'x'
#             y.save()

#         onschedule_obj.schedule_name = 'x'
#         onschedule_obj.save()
        
#         settings.TIME_ZONE  # 'UTC'

#         mv.report_datetime = make_aware(datetime.datetime.combine(a[1], 
#                             mv.report_datetime.time()))
#         mv.save()

#         for r in app_not_new:
#             g = Appointment.objects.get(subject_identifier=r.subject_identifier, visit_code=r.visit_code,
#                                        schedule_name=schedule_name)
#             g.appt_status = r.appt_status
#             g.appt_datetime = r.appt_datetime
#             g.appt_reason = r.appt_reason
#             g.comment = r.comment
#             g.save()

#             try:
#                 mm = MaternalVisit.objects.get(appointment=r)
#             except MaternalVisit.DoesNotExist:
#                 pass
#             else:
#                 mm.appointment = g
#                 mm.save_base(raw=True)

#             r.delete()

#         onschedule_obj.delete()


[]